## 3) Run optimization 

In this notebook we use the previously computed features to fit the cell model parameters and save the results.

We can run optimization on a subset of samples/feature sets.

In [ ]:
%load_ext autoreload
%autoreload

#!nrnivmodl mechanisms
import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import matplotlib.pyplot as plt
import MEAutility as mu
import json
import numpy
import time
import numpy as np
import LFPy
from pathlib import Path
import pandas as pd
import os
from ipyparallel import Client

%matplotlib notebook

In [ ]:
import l5pc_model
import l5pc_evaluator
import l5pc_plot

### Set up ipyparallel processing

In [ ]:
try:
    rc = Client() # profile=os.getenv('IPYTHON_PROFILE')
    lview = rc.load_balanced_view()
    
    def mapper(func, it):
        start_time = time.time()
        ret = lview.map_sync(func, it)
        print(f'Generation {it} took {time.time() - start_time} s')
        return ret
except:
    mapper = None

if mapper is not None:
    print(f'IPyparallel started successfully with {len(rc)} parallel processes')

In [ ]:
feature_sets = ['extra']  # ['soma', 'bap', 'extra']
sample_ids = [0] # [0, ..., n_samples]

# offspring_size = 250
# max_ngen = 50
# channels = None
offspring_size = 2
max_ngen = 2
channels = 'map'

# define map_function
map_function=mapper

In [ ]:
random_params_file = 'config/params/smart_random.csv'
random_params = pd.read_csv(random_params_file, index_col='index')
params = random_params.iloc[0].to_dict()

In [ ]:
prep = l5pc_evaluator.prepare_optimization('extra', 0, offspring_size=offspring_size, channels='map', 
                                           probe_type='linear', map_function=map_function)
opt = prep['optimisation']
evaluator = prep['evaluator']
fitness_calculator = prep['objectives_calculator']
fitness_protocols = prep['protocols']

In [ ]:
out = l5pc_evaluator.run_optimization(feature_set='extra', sample_id=0, opt=opt, 
                                      max_ngen=max_ngen, channels=channels)

In [ ]:
final_pop = out['final_pop']
halloffame = out['halloffame']
log = out['log']
hist = out['hist']

In [ ]:
best_params = evaluator.param_dict(halloffame[0])
t_start = time.time()
best_responses = evaluator.run_protocols(protocols=fitness_protocols.values(), param_values=best_params)
t_stop = time.time()
print(f'Time one protocol: {t_stop - t_start} s')

In [ ]:
original_responses = evaluator.run_protocols(protocols=fitness_protocols.values(), param_values=params)

In [ ]:
def plot_responses(responses):
    resp_no_mea = {}
    for (resp_name, response) in sorted(responses.items()):
        if 'MEA' not in resp_name:
            resp_no_mea[resp_name] = response
    fig, axes = plt.subplots(len(resp_no_mea), figsize=(10,10))
    for index, (resp_name, response) in enumerate(sorted(resp_no_mea.items())):
        axes[index].plot(response['time'], response['voltage'], label=resp_name)
        axes[index].set_title(resp_name)
    fig.tight_layout()
    fig.show()

In [ ]:
plot_responses(best_responses)

In [ ]:
plot_responses(original_responses)

In [ ]:
for param in best_params.keys():
    print(param, params[param],  best_params[param])